## Simple flow of listing and querying Kafka Topics as Iceberg tables

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

In [ ]:
%%sql

USE isk.isk

In [ ]:
%%sql

show tables;

In [ ]:
%%sql

DESCRIBE transactions;

In [10]:
%%sql

SELECT count(*) from isk.isk.transactions.files;

count(1)
112


In [11]:
%%sql

SELECT *
FROM isk.isk.transactions.files;

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://transactions/iceberg_data/10000/0-945000-955000.avro,AVRO,0,Row(transactionTime_hour=484871),10000,100024,None,None,None,None,None,None,None,None,None,0,"Row(amount=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), itemCode=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), kafka_offset=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), kafka_ts=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), paymentTermCode=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), storeId=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), transactionTime=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None))"
0,s3://transactions/iceberg_data/10000/0-936000-946000.avro,AVRO,0,Row(transactionTime_hour=484870),10000,100024,None,None,None,None,None,None,None,None,None,0,"Row(amount=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), itemCode=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), kafka_offset=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), kafka_ts=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), paymentTermCode=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), storeId=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), transactionTime=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None))"
0,s3://transactions/iceberg_data/10000/0-927000-937000.avro,AVRO,0,Row(transactionTime_hour=484869),10000,100024,None,None,None,None,None,None,None,None,None,0,"Row(amount=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), itemCode=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), kafka_offset=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), kafka_ts=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), paymentTermCode=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), storeId=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), transactionTime=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None))"
0,s3://transactions/iceberg_data/10000/0-918000-928000.avro,AVRO,0,Row(transactionTime_hour=484868),10000,100024,None,None,None,None,None,None,None,None,None,0,"Row(amount=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), itemCode=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), kafka_offset=Row(column_size=None, value_co

In [ ]:
%%sql
-- Total taken in each currency - for single day - 21/04/2025.    
    
select 
    round(sum(t.amount),2) as total_taken,
    p.currency  
from 
    transactions t 
join 
    payment_terms p 
on 
    t.paymenttermcode=p.termcode
where 
    t.transactiontime between '2025-04-21 00:00:00' AND '2025-04-21 23:59:59'
group by 
    p.currency 
order by 
    p.currency asc;

In [ ]:
%%sql
-- Total taken in each currency - without time bounds
    
select 
    round(sum(t.amount),2) as total_taken,
    p.currency  
from 
    transactions t 
join 
    payment_terms p 
on 
    t.paymenttermcode=p.termcode
--where 
--    t.transactiontime between '2025-04-21 00:00:00' AND '2025-04-21 23:59:59'
group by 
    p.currency 
order by 
    p.currency asc;